# 서울시 상권분석 데이터 수집 및 전처리 (개선 버전)
- API 에러 처리 강화
- 로깅 추가
- 데이터 검증 로직 추가

In [ ]:
import requests
import dotenv
import os
import json
import pandas as pd
from datetime import datetime
import logging

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# 환경변수 로드
dotenv.load_dotenv()
key = os.getenv("SEOUL_API")

if not key:
    logger.error("API 키를 찾을 수 없습니다. .env 파일을 확인하세요.")
    raise ValueError("SEOUL_API 키가 설정되지 않았습니다.")

logger.info("API 키 로드 완료")

In [ ]:
# API 호출 함수 (에러 처리 강화)
def fetch_seoul_data(api_key, start_idx=1, end_idx=1000, max_retries=3):
    """
    서울시 상권분석 데이터 API 호출
    
    Args:
        api_key: API 인증키
        start_idx: 시작 인덱스
        end_idx: 종료 인덱스
        max_retries: 최대 재시도 횟수
    
    Returns:
        dict: API 응답 데이터
    """
    url = f'http://openapi.seoul.go.kr:8088/{api_key}/json/VwsmMegaSelngW/{start_idx}/{end_idx}/'
    
    for attempt in range(max_retries):
        try:
            logger.info(f"API 호출 시도 {attempt + 1}/{max_retries}")
            response = requests.get(url, timeout=30)
            
            logger.info(f'접속주소: {response.url}')
            logger.info(f'응답코드: {response.status_code}')
            
            if response.status_code == 200:
                result = response.json()
                
                # API 오류 체크
                if 'RESULT' in result and result['RESULT']['CODE'] != 'INFO-000':
                    logger.error(f"API 오류: {result['RESULT']['MESSAGE']}")
                    return None
                
                return result
            else:
                logger.warning(f"HTTP 오류: {response.status_code}")
                
        except requests.Timeout:
            logger.warning(f"타임아웃 발생 (시도 {attempt + 1}/{max_retries})")
        except requests.RequestException as e:
            logger.error(f"요청 오류: {e}")
        except json.JSONDecodeError:
            logger.error("JSON 파싱 오류")
    
    logger.error("API 호출 실패")
    return None

# 데이터 가져오기
result = fetch_seoul_data(key)

if result and 'VwsmMegaSelngW' in result:
    data = result['VwsmMegaSelngW']['row']
    logger.info(f'데이터 개수: {len(data)}')
    print(json.dumps(data[0], indent=4, ensure_ascii=False))
else:
    logger.error("데이터를 가져올 수 없습니다.")
    data = []

In [ ]:
# 컬럼명 매핑 (상수로 분리)
COLUMN_MAPPING = {
    "STDR_YYQU_CD": "기준_년분기_코드",
    "MEGA_CD": "서울시_코드",
    "MEGA_CD_NM": "서울시_코드_명",
    "SVC_INDUTY_CD": "서비스_업종_코드",
    "SVC_INDUTY_CD_NM": "서비스_업종_코드_명",
    "THSMON_SELNG_AMT": "당월_매출_금액",
    "THSMON_SELNG_CO": "당월_매출_건수",
    "MDWK_SELNG_AMT": "주중_매출_금액",
    "WKEND_SELNG_AMT": "주말_매출_금액",
    "MON_SELNG_AMT": "월요일_매출_금액",
    "TUES_SELNG_AMT": "화요일_매출_금액",
    "WED_SELNG_AMT": "수요일_매출_금액",
    "THUR_SELNG_AMT": "목요일_매출_금액",
    "FRI_SELNG_AMT": "금요일_매출_금액",
    "SAT_SELNG_AMT": "토요일_매출_금액",
    "SUN_SELNG_AMT": "일요일_매출_금액",
    "TMZON_00_06_SELNG_AMT": "시간대_00~06_매출_금액",
    "TMZON_06_11_SELNG_AMT": "시간대_06~11_매출_금액",
    "TMZON_11_14_SELNG_AMT": "시간대_11~14_매출_금액",
    "TMZON_14_17_SELNG_AMT": "시간대_14~17_매출_금액",
    "TMZON_17_21_SELNG_AMT": "시간대_17~21_매출_금액",
    "TMZON_21_24_SELNG_AMT": "시간대_21~24_매출_금액",
    "ML_SELNG_AMT": "남성_매출_금액",
    "FML_SELNG_AMT": "여성_매출_금액",
    "AGRDE_10_SELNG_AMT": "연령대_10_매출_금액",
    "AGRDE_20_SELNG_AMT": "연령대_20_매출_금액",
    "AGRDE_30_SELNG_AMT": "연령대_30_매출_금액",
    "AGRDE_40_SELNG_AMT": "연령대_40_매출_금액",
    "AGRDE_50_SELNG_AMT": "연령대_50_매출_금액",
    "AGRDE_60_ABOVE_SELNG_AMT": "연령대_60_이상_매출_금액",
    "MDWK_SELNG_CO": "주중_매출_건수",
    "WKEND_SELNG_CO": "주말_매출_건수",
    "MON_SELNG_CO": "월요일_매출_건수",
    "TUES_SELNG_CO": "화요일_매출_건수",
    "WED_SELNG_CO": "수요일_매출_건수",
    "THUR_SELNG_CO": "목요일_매출_건수",
    "FRI_SELNG_CO": "금요일_매출_건수",
    "SAT_SELNG_CO": "토요일_매출_건수",
    "SUN_SELNG_CO": "일요일_매출_건수",
    "TMZON_00_06_SELNG_CO": "시간대_00~06_매출_건수",
    "TMZON_06_11_SELNG_CO": "시간대_06~11_매출_건수",
    "TMZON_11_14_SELNG_CO": "시간대_11~14_매출_건수",
    "TMZON_14_17_SELNG_CO": "시간대_14~17_매출_건수",
    "TMZON_17_21_SELNG_CO": "시간대_17~21_매출_건수",
    "TMZON_21_24_SELNG_CO": "시간대_21~24_매출_건수",
    "ML_SELNG_CO": "남성_매출_건수",
    "FML_SELNG_CO": "여성_매출_건수",
    "AGRDE_10_SELNG_CO": "연령대_10_매출_건수",
    "AGRDE_20_SELNG_CO": "연령대_20_매출_건수",
    "AGRDE_30_SELNG_CO": "연령대_30_매출_건수",
    "AGRDE_40_SELNG_CO": "연령대_40_매출_건수",
    "AGRDE_50_SELNG_CO": "연령대_50_매출_건수",
    "AGRDE_60_ABOVE_SELNG_CO": "연령대_60_이상_매출_건수"
}

if data:
    # DataFrame 생성
    df = pd.DataFrame(data)
    
    # 컬럼명 변경
    df.rename(columns=COLUMN_MAPPING, inplace=True)
    
    # 인덱스 설정
    df.set_index('기준_년분기_코드', inplace=True)
    
    # 불필요한 컬럼 제거
    columns_to_drop = ['서울시_코드', '서울시_코드_명', '서비스_업종_코드']
    df.drop(columns=columns_to_drop, inplace=True, errors='ignore')
    
    # 데이터 타입 변환 (매출 관련 컬럼을 숫자로)
    numeric_columns = [col for col in df.columns if '매출' in col or '건수' in col]
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    logger.info(f"데이터 처리 완료: {len(df)} 행")
    print("\n=== 데이터 요약 ===")
    print(df.info())
    print("\n=== 처음 5개 행 ===")
    print(df.head())
else:
    logger.error("처리할 데이터가 없습니다.")
    df = pd.DataFrame()

In [ ]:
# 데이터 저장
if not df.empty:
    try:
        # 타임스탬프 추가
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        
        # CSV 저장
        csv_filename = f'seoul_store_{timestamp}.csv'
        df.to_csv(csv_filename, encoding='cp949')
        logger.info(f"CSV 파일 저장 완료: {csv_filename}")
        
        # 원본 파일명으로도 저장 (호환성 유지)
        df.to_csv('seoul_store.csv', encoding='cp949')
        logger.info("CSV 파일 저장 완료: seoul_store.csv")
        
    except Exception as e:
        logger.error(f"파일 저장 오류: {e}")
else:
    logger.warning("저장할 데이터가 없습니다.")